In [24]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.figure_factory as ff
import plotly.express as px
from linearmodels.iv import IV2SLS
import itertools
import statsmodels.api as sm
import datetime
import geopandas as gpd
import warnings
import array_to_latex as a2l

warnings.filterwarnings('ignore')

In Glaser, Gorback, and Redding's paper, they assumed that the effect of changes in mobility will be reflected on the infection rate in a week. As a result, I take the average of changes in mobility and COVID cases in this analysis. 

In [25]:
ca = pd.read_csv('output/data/CA.csv', dtype={'fips': 'str'})
ca['date'] = pd.to_datetime(ca['date'])
ca['ln_cases'] = np.log(ca['daily_cases_100k'])
ca.replace([np.inf, -np.inf], 0, inplace=True)
ca['ln_cases'] = ca['ln_cases'].astype('float64')
ca['ln_deaths'] = np.log(ca['daily_deaths_100k'])
ca.replace([np.inf, -np.inf], 0, inplace=True)
ca['ln_deaths'] = ca['ln_deaths'].astype('float64')
ca['Year_week'] = ca['date'].dt.strftime('%Y-%U')
ca.drop(columns=['index'], inplace=True)
ca.head(5)

,date,county,state,fips,cases,deaths,daily_cases,daily_deaths,retail_and_recreation,grocery_and_pharmacy,...,bachelor_or_higher,high_school,less_than_high_school,poverty_rate,daily_cases_100k,daily_deaths_100k,party_DEMOCRAT,ln_cases,ln_deaths,Year_week
0,2020-02-15,Los Angeles,California,06037,1,0.0,0.0,0.0,1.0,0.0,...,34.044369,20.390964,19.957163,13.2,0.0,0.0,1,0.0,0.0,2020-06
1,2020-02-15,Orange,California,06059,1,0.0,0.0,0.0,0.0,0.0,...,42.115793,17.244207,13.388919,9.0,0.0,0.0,1,0.0,0.0,2020-06
2,2020-02-15,San Diego,California,06073,1,0.0,0.0,0.0,3.0,-1.0,...,40.263250,18.196428,11.684829,9.5,0.0,0.0,1,0.0,0.0,2020-06
3,2020-02-15,San Francisco,California,06075,2,0.0,0.0,0.0,6.0,2.0,...,59.516756,11.373371,11.219663,10.0,0.0,0.0,1,0.0,0.0,2020-06
4,2020-02-15,Santa Clara,California,06085,2,0.0,0.0,0.0,-3.0,-3.0,...,54.391969,13.699706,10.812734,6.6,0.0,0.0,1,0.0,0.0,2020-06


In [26]:
sa = pd.read_csv('output/data/SA.csv', dtype={'fips': 'str'})
sa['date'] = pd.to_datetime(sa['date'])
sa['ln_cases'] = np.log(sa['daily_cases_100k'])
sa.replace([np.inf, -np.inf], 0, inplace=True)
sa['ln_cases'] = sa['ln_cases'].astype('float64')
sa['ln_deaths'] = np.log(sa['daily_deaths_100k'])
sa.replace([np.inf, -np.inf], 0, inplace=True)
sa['ln_deaths'] = sa['ln_deaths'].astype('float64')
sa['Year_week'] = sa['date'].dt.strftime('%Y-%U')
sa.drop(columns=['index'], inplace=True)
sa.head(5)

,date,county,state,fips,cases,deaths,daily_cases,daily_deaths,retail_and_recreation,grocery_and_pharmacy,...,bachelor_or_higher,high_school,less_than_high_school,poverty_rate,daily_cases_100k,daily_deaths_100k,party_DEMOCRAT,ln_cases,ln_deaths,Year_week
0,2020-03-06,Charleston,South Carolina,45019,1,0.0,1.0,0.0,9.0,5.0,...,46.738602,20.984812,7.114868,11.9,0.256896,0.0,1,-1.359082,0.0,2020-09
1,2020-03-06,Kershaw,South Carolina,45055,1,0.0,1.0,0.0,10.0,10.0,...,21.189354,35.660776,10.230027,14.4,1.572253,0.0,1,0.452510,0.0,2020-09
2,2020-03-07,Charleston,South Carolina,45019,1,0.0,0.0,0.0,7.0,5.0,...,46.738602,20.984812,7.114868,11.9,0.000000,0.0,1,0.000000,0.0,2020-09
3,2020-03-07,Kershaw,South Carolina,45055,1,0.0,0.0,0.0,11.0,14.0,...,21.189354,35.660776,10.230027,14.4,0.000000,0.0,1,0.000000,0.0,2020-09
4,2020-03-08,Charleston,South Carolina,45019,1,0.0,0.0,0.0,11.0,5.0,...,46.738602,20.984812,7.114868,11.9,0.000000,0.0,1,0.000000,0.0,2020-10


In [27]:
oh = pd.read_csv('output/data/OH.csv', dtype={'fips': 'str'})
oh['date'] = pd.to_datetime(oh['date'])
oh['ln_cases'] = np.log(oh['daily_cases_100k'])
oh.replace([np.inf, -np.inf], 0, inplace=True)
oh['ln_cases'] = oh['ln_cases'].astype('float64')
oh['ln_deaths'] = np.log(oh['daily_deaths_100k'])
oh.replace([np.inf, -np.inf], 0, inplace=True)
oh['ln_deaths'] = oh['ln_deaths'].astype('float64')
oh['Year_week'] = oh['date'].dt.strftime('%Y-%U')
oh.drop(columns=['index'], inplace=True)
oh.head(5)

,date,county,state,fips,cases,deaths,daily_cases,daily_deaths,retail_and_recreation,grocery_and_pharmacy,...,bachelor_or_higher,high_school,less_than_high_school,poverty_rate,daily_cases_100k,daily_deaths_100k,party_DEMOCRAT,ln_cases,ln_deaths,Year_week
0,2020-03-09,Cuyahoga,Ohio,39035,3,0.0,3.0,0.0,9.0,8.0,...,34.374877,27.336715,9.283024,15.3,0.238869,0.0,1,-1.431842,0.0,2020-10
1,2020-03-10,Cuyahoga,Ohio,39035,3,0.0,0.0,0.0,3.0,4.0,...,34.374877,27.336715,9.283024,15.3,0.000000,0.0,1,0.000000,0.0,2020-10
2,2020-03-11,Cuyahoga,Ohio,39035,3,0.0,0.0,0.0,5.0,8.0,...,34.374877,27.336715,9.283024,15.3,0.000000,0.0,1,0.000000,0.0,2020-10
3,2020-03-11,Stark,Ohio,39151,1,0.0,1.0,0.0,9.0,15.0,...,23.584225,38.175058,7.703114,13.2,0.266549,0.0,1,-1.322196,0.0,2020-10
4,2020-03-12,Cuyahoga,Ohio,39035,3,0.0,0.0,0.0,5.0,29.0,...,34.374877,27.336715,9.283024,15.3,0.000000,0.0,1,0.000000,0.0,2020-10


In [28]:
md = pd.read_csv('output/data/MD.csv', dtype={'fips': 'str'})
md['date'] = pd.to_datetime(md['date'])
md['ln_cases'] = np.log(md['daily_cases_100k'])
md.replace([np.inf, -np.inf], 0, inplace=True)
md['ln_cases'] = md['ln_cases'].astype('float64')
md['ln_deaths'] = np.log(md['daily_deaths_100k'])
md.replace([np.inf, -np.inf], 0, inplace=True)
md['ln_deaths'] = md['ln_deaths'].astype('float64')
md['Year_week'] = md['date'].dt.strftime('%Y-%U')
md.drop(columns=['index'], inplace=True)
md.head(5)

,date,county,state,fips,cases,deaths,daily_cases,daily_deaths,retail_and_recreation,grocery_and_pharmacy,...,bachelor_or_higher,high_school,less_than_high_school,poverty_rate,daily_cases_100k,daily_deaths_100k,party_DEMOCRAT,ln_cases,ln_deaths,Year_week
0,2020-03-05,Montgomery,Maryland,24031,3,0.0,3.0,0.0,4.0,4.0,...,59.781093,13.129228,8.768636,6.7,0.288429,0.0,1,-1.243305,0.0,2020-09
1,2020-03-06,Montgomery,Maryland,24031,3,0.0,0.0,0.0,-1.0,3.0,...,59.781093,13.129228,8.768636,6.7,0.000000,0.0,1,0.000000,0.0,2020-09
2,2020-03-07,Montgomery,Maryland,24031,3,0.0,0.0,0.0,1.0,2.0,...,59.781093,13.129228,8.768636,6.7,0.000000,0.0,1,0.000000,0.0,2020-09
3,2020-03-08,Harford,Maryland,24025,1,0.0,1.0,0.0,11.0,2.0,...,37.698215,26.170423,6.267963,6.2,0.399537,0.0,1,-0.917450,0.0,2020-10
4,2020-03-08,Montgomery,Maryland,24031,4,0.0,1.0,0.0,2.0,2.0,...,59.781093,13.129228,8.768636,6.7,0.096143,0.0,1,-2.341917,0.0,2020-10


In [29]:
me = pd.read_csv('output/data/ME.csv', dtype={'fips': 'str'})
me['date'] = pd.to_datetime(me['date'])
me['ln_cases'] = np.log(me['daily_cases_100k'])
me.replace([np.inf, -np.inf], 0, inplace=True)
me['ln_cases'] = me['ln_cases'].astype('float64')
me['ln_deaths'] = np.log(me['daily_deaths_100k'])
me.replace([np.inf, -np.inf], 0, inplace=True)
me['ln_deaths'] = me['ln_deaths'].astype('float64')
me['Year_week'] = me['date'].dt.strftime('%Y-%U')
me.drop(columns=['index'], inplace=True)
me.head(5)

,date,county,state,fips,cases,deaths,daily_cases,daily_deaths,retail_and_recreation,grocery_and_pharmacy,...,bachelor_or_higher,high_school,less_than_high_school,poverty_rate,daily_cases_100k,daily_deaths_100k,party_DEMOCRAT,ln_cases,ln_deaths,Year_week
0,2020-03-12,Androscoggin,Maine,23001,1,0.0,1.0,0.0,12.0,36.0,...,22.921006,35.568111,8.211831,11.7,0.932549,0.0,1,-0.069834,0.0,2020-10
1,2020-03-13,Androscoggin,Maine,23001,1,0.0,0.0,0.0,-6.0,21.0,...,22.921006,35.568111,8.211831,11.7,0.000000,0.0,1,0.000000,0.0,2020-10
2,2020-03-13,Cumberland,Maine,23005,1,0.0,1.0,0.0,-6.0,25.0,...,49.809385,20.368462,4.333531,8.6,0.344855,0.0,1,-1.064631,0.0,2020-10
3,2020-03-14,Androscoggin,Maine,23001,1,0.0,0.0,0.0,-4.0,16.0,...,22.921006,35.568111,8.211831,11.7,0.000000,0.0,1,0.000000,0.0,2020-10
4,2020-03-14,Cumberland,Maine,23005,2,0.0,1.0,0.0,-7.0,19.0,...,49.809385,20.368462,4.333531,8.6,0.344855,0.0,1,-1.064631,0.0,2020-10


In [30]:
ny = pd.read_csv('output/data/NY.csv', dtype={'fips': 'str'})
ny['date'] = pd.to_datetime(ny['date'])
ny['ln_cases'] = np.log(ny['daily_cases_100k'])
ny.replace([np.inf, -np.inf], 0, inplace=True)
ny['ln_cases'] = ny['ln_cases'].astype('float64')
ny['Year_week'] = ny['date'].dt.strftime('%Y-%U')
ny.drop(columns=['index'], inplace=True)
ny.head(5)

,date,county,state,fips,cases,deaths,daily_cases,daily_deaths,retail_and_recreation,grocery_and_pharmacy,...,some_college,bachelor_or_higher,high_school,less_than_high_school,poverty_rate,daily_cases_100k,daily_deaths_100k,party_DEMOCRAT,ln_cases,Year_week
0,2020-03-04,Westchester,New York,36119,9,0.0,9.0,0.0,7.0,6.0,...,19.767242,50.649474,18.434973,11.148311,7.6,0.921757,0.0,1,-0.081473,2020-09
1,2020-03-05,Nassau,New York,36059,1,0.0,1.0,0.0,5.0,7.0,...,22.421977,47.481852,21.799113,8.297058,5.7,0.073456,0.0,1,-2.611062,2020-09
2,2020-03-05,Westchester,New York,36119,17,0.0,8.0,0.0,6.0,6.0,...,19.767242,50.649474,18.434973,11.148311,7.6,0.819340,0.0,1,-0.199257,2020-09
3,2020-03-06,Nassau,New York,36059,4,0.0,3.0,0.0,-1.0,4.0,...,22.421977,47.481852,21.799113,8.297058,5.7,0.220369,0.0,1,-1.512450,2020-09
4,2020-03-06,Rockland,New York,36087,2,0.0,2.0,0.0,-2.0,3.0,...,24.770103,41.964469,22.121321,11.144107,14.4,0.613427,0.0,1,-0.488694,2020-09


In [31]:
pa = pd.read_csv('output/data/PA.csv', dtype={'fips': 'str'})
pa['date'] = pd.to_datetime(pa['date'])
pa['ln_cases'] = np.log(pa['daily_cases_100k'])
pa.replace([np.inf, -np.inf], 0, inplace=True)
pa['ln_cases'] = pa['ln_cases'].astype('float64')
pa['ln_deaths'] = np.log(pa['daily_deaths_100k'])
pa['Year_week'] = pa['date'].dt.strftime('%Y-%U')
pa.drop(columns=['index'], inplace=True)
pa.head(5)

,date,county,state,fips,cases,deaths,daily_cases,daily_deaths,retail_and_recreation,grocery_and_pharmacy,...,bachelor_or_higher,high_school,less_than_high_school,poverty_rate,daily_cases_100k,daily_deaths_100k,party_DEMOCRAT,ln_cases,ln_deaths,Year_week
0,2020-03-06,Delaware,Pennsylvania,42045,1,0.0,1.0,0.0,-1.0,1.0,...,40.547734,27.916247,6.562482,9.3,0.177338,0.0,1,-1.729696,-inf,2020-09
1,2020-03-06,Wayne,Pennsylvania,42127,1,0.0,1.0,0.0,0.0,9.0,...,19.591691,44.293596,9.472962,10.5,1.953201,0.0,1,0.669470,-inf,2020-09
2,2020-03-07,Delaware,Pennsylvania,42045,1,0.0,0.0,0.0,7.0,9.0,...,40.547734,27.916247,6.562482,9.3,0.000000,0.0,1,0.000000,-inf,2020-09
3,2020-03-07,Montgomery,Pennsylvania,42091,2,0.0,2.0,0.0,4.0,7.0,...,50.747544,22.763877,5.342989,5.6,0.244122,0.0,1,-1.410089,-inf,2020-09
4,2020-03-07,Wayne,Pennsylvania,42127,1,0.0,0.0,0.0,11.0,8.0,...,19.591691,44.293596,9.472962,10.5,0.000000,0.0,1,0.000000,-inf,2020-09


In [32]:
ca_missing = ca.isnull().sum() / len(ca)
md_missing = md.isnull().sum() / len(md)
me_missing = me.isnull().sum() / len(me)
ny_missing = ny.isnull().sum() / len(ny)
oh_missing = oh.isnull().sum() / len(oh)
pa_missing = pa.isnull().sum() / len(pa)
sa_missing = sa.isnull().sum() / len(sa)
missings = pd.concat([ca_missing, md_missing, me_missing, ny_missing, oh_missing, pa_missing, sa_missing], axis=1)
missings.columns = ['ca', 'md', 'me', 'ny', 'oh', 'pa', 'sa']
missings

,ca,md,me,ny,oh,pa,sa
date,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
county,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
state,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
fips,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
cases,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
deaths,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
daily_cases,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
daily_deaths,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
retail_and_recreation,0.063220,0.069866,0.105458,0.076692,0.143593,0.117754,0.296080
grocery_and_pharmacy,0.118056,0.173462,0.266995,0.126569,0.220162,0.147820,0.254591


In [33]:
ny

,date,county,state,fips,cases,deaths,daily_cases,daily_deaths,retail_and_recreation,grocery_and_pharmacy,...,some_college,bachelor_or_higher,high_school,less_than_high_school,poverty_rate,daily_cases_100k,daily_deaths_100k,party_DEMOCRAT,ln_cases,Year_week
0,2020-03-04,Westchester,New York,36119,9,0.0,9.0,0.0,7.0,6.0,...,19.767242,50.649474,18.434973,11.148311,7.6,0.921757,0.0,1,-0.081473,2020-09
1,2020-03-05,Nassau,New York,36059,1,0.0,1.0,0.0,5.0,7.0,...,22.421977,47.481852,21.799113,8.297058,5.7,0.073456,0.0,1,-2.611062,2020-09
2,2020-03-05,Westchester,New York,36119,17,0.0,8.0,0.0,6.0,6.0,...,19.767242,50.649474,18.434973,11.148311,7.6,0.819340,0.0,1,-0.199257,2020-09
3,2020-03-06,Nassau,New York,36059,4,0.0,3.0,0.0,-1.0,4.0,...,22.421977,47.481852,21.799113,8.297058,5.7,0.220369,0.0,1,-1.512450,2020-09
4,2020-03-06,Rockland,New York,36087,2,0.0,2.0,0.0,-2.0,3.0,...,24.770103,41.964469,22.121321,11.144107,14.4,0.613427,0.0,1,-0.488694,2020-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52504,2022-10-15,Washington,New York,36115,14751,128.0,15.0,0.0,21.0,17.0,...,29.075027,20.587974,39.012061,11.324938,11.2,24.104130,0.0,1,3.182383,2022-41
52505,2022-10-15,Wayne,New York,36117,20591,197.0,21.0,0.0,15.0,1.0,...,33.376923,24.438462,32.521538,9.663077,9.7,22.964372,0.0,1,3.133944,2022-41
52506,2022-10-15,Westchester,New York,36119,308361,2844.0,186.0,0.0,-10.0,-3.0,...,19.767242,50.649474,18.434973,11.148311,7.6,19.049648,0.0,1,2.947049,2022-41
52507,2022-10-15,Wyoming,New York,36121,9501,88.0,8.0,0.0,38.0,NaN,...,32.509120,18.109039,37.788812,11.593028,9.6,19.506010,0.0,1,2.970723,2022-41


In [34]:
ny_week = ny.groupby(['Year_week', 'fips']).mean().reset_index()
ny_week['ln_cases_lag'] = ny.groupby(['Year_week', 'fips'])['ln_cases'].shift(1)
ny_week['transit_stations_lag'] = ny.groupby(['Year_week', 'fips'])['transit_stations'].shift(1)
ny_week['parks_lag'] = ny.groupby(['Year_week', 'fips'])['parks'].shift(1)
ny_week['residential_lag'] = ny.groupby(['Year_week', 'fips'])['residential'].shift(1)
ny_week['retail_and_recreation_lag'] = ny.groupby(['Year_week', 'fips'])['retail_and_recreation'].shift(1)
ny_week['workplaces_lag'] = ny.groupby(['Year_week', 'fips'])['workplaces'].shift(1)

In [35]:
# Perform linear regression of U_t on a constant term and U_{t-1}
X2 = ny_week[['ln_cases_lag']].dropna()
X2 = sm.add_constant(X2)  # Add a constant term to the model
y2 = ny_week.loc[X2.index, 'ln_cases']

model2 = sm.OLS(y2, X2).fit()

In [36]:
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               ln_cases   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     65.89
Date:                Wed, 17 May 2023   Prob (F-statistic):           5.66e-16
Time:                        22:41:04   Log-Likelihood:                -11163.
No. Observations:                6531   AIC:                         2.233e+04
Df Residuals:                    6529   BIC:                         2.234e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            2.5938      0.022    120.201      0.000       2.551       2.636
ln_cases_lag    -0.1274      0.016     -8.117      0.000      -0.158      -0.097
==============================================================================
Omnibus:                      648.498   Durbin-Watson:                   0.316
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              235.193
Skew:                          -0.222   Prob(JB):                     8.48e-52
Kurtosis:                       2.184   Cond. No.                         2.31
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [37]:
ny_week

,Year_week,fips,cases,deaths,daily_cases,daily_deaths,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,...,daily_cases_100k,daily_deaths_100k,party_DEMOCRAT,ln_cases,ln_cases_lag,transit_stations_lag,parks_lag,residential_lag,retail_and_recreation_lag,workplaces_lag
0,2020-09,36059,3.000000,0.000000,1.333333,0.000000,3.333333,6.333333,15.333333,-5.000000,...,0.097942,0.000000,1.0,-1.374504,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-09,36087,2.000000,0.000000,1.000000,0.000000,1.500000,5.500000,34.000000,2.500000,...,0.306714,0.000000,1.0,-0.244347,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-09,36091,2.000000,0.000000,2.000000,0.000000,13.000000,10.000000,24.000000,18.000000,...,0.883982,0.000000,1.0,-0.123319,-0.081473,-1.0,28.0,-1.0,7.0,2.0
3,2020-09,36119,32.000000,0.000000,17.250000,0.000000,3.000000,2.250000,20.500000,-1.500000,...,1.766701,0.000000,1.0,0.379495,-2.611062,-2.0,25.0,0.0,5.0,4.0
4,2020-10,36001,3.000000,0.000000,1.666667,0.000000,-6.666667,28.000000,23.000000,-8.000000,...,0.538710,0.000000,1.0,-0.155682,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7659,2022-41,36115,14711.285714,128.000000,13.142857,0.000000,15.142857,17.857143,NaN,NaN,...,21.119809,0.000000,1.0,3.028969,1.482138,-21.0,156.0,10.0,-15.0,-41.0
7660,2022-41,36117,20532.285714,196.571429,17.857143,0.142857,10.285714,-0.714286,NaN,NaN,...,19.527528,0.156220,1.0,2.944390,1.393480,NaN,NaN,6.0,-11.0,-32.0
7661,2022-41,36119,307791.571429,2840.857143,179.714286,1.000000,-11.857143,-2.285714,43.428571,-20.285714,...,18.405881,0.102417,1.0,2.896201,1.195601,-47.0,115.0,14.0,-17.0,-43.0
7662,2022-41,36121,9479.571429,88.000000,7.000000,0.000000,35.250000,NaN,NaN,NaN,...,17.067759,0.000000,1.0,2.789507,1.037273,-23.0,267.0,7.0,-8.0,-35.0


In [38]:
X3 = ny_week[['transit_stations_lag', 'parks_lag', 'residential_lag', 'retail_and_recreation_lag', 'workplaces_lag',
              'unemployment_rate', 'some_college', 'bachelor_or_higher', 'high_school', 'less_than_high_school',
              'population_density',
              'poverty_rate', 'party_DEMOCRAT']].dropna()
X3 = sm.add_constant(X3)  # Add a constant term to the model
y3 = ny_week.loc[X3.index, 'ln_cases']
model3 = sm.OLS(y3, X3).fit()
model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               ln_cases   R-squared:                       0.326
Model:                            OLS   Adj. R-squared:                  0.322
Method:                 Least Squares   F-statistic:                     75.87
Date:                Wed, 17 May 2023   Prob (F-statistic):          3.14e-139
Time:                        22:41:04   Log-Likelihood:                -2718.4
No. Observations:                1737   AIC:                             5461.
Df Residuals:                    1725   BIC:                             5526.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
transit_stations_lag          0.0061      0.003      2.357      0.019       0.001       0.011
parks_lag                    -0.0013      0.000     -2.583      0.010      -0.002      -0.000
residential_lag               0.0147      0.011      1.289      0.197      -0.008       0.037
retail_and_recreation_lag    -0.0223      0.003     -7.626      0.000      -0.028      -0.017
workplaces_lag                0.0164      0.005      3.248      0.001       0.007       0.026
unemployment_rate            -0.5194      0.023    -22.848      0.000      -0.564      -0.475
some_college                  0.1226      0.011     10.978      0.000       0.101       0.145
bachelor_or_higher            0.0080      0.004      2.054      0.040       0.000       0.016
high_school                  -0.0091      0.010     -0.890      0.374      -0.029       0.011
less_than_high_school         0.0855      0.022      3.865      0.000       0.042       0.129
population_density            0.0007   8.07e-05      8.838      0.000       0.001       0.001
poverty_rate                  0.0432      0.013      3.405      0.001       0.018       0.068
party_DEMOCRAT                0.0021      0.000     10.678      0.000       0.002       0.002
==============================================================================
Omnibus:                       25.186   Durbin-Watson:                   0.554
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               21.363
Skew:                           0.206   Prob(JB):                     2.30e-05
Kurtosis:                       2.646   Cond. No.                     5.90e+18
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.92e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [39]:
ca_week = ca.groupby(['Year_week', 'fips']).mean().reset_index()
ca_week['ln_cases_lag'] = ca.groupby(['Year_week', 'fips'])['ln_cases'].shift(1)
ca_week['transit_stations_lag'] = ca.groupby(['Year_week', 'fips'])['transit_stations'].shift(1)
ca_week['parks_lag'] = ca.groupby(['Year_week', 'fips'])['parks'].shift(1)
ca_week['residential_lag'] = ca.groupby(['Year_week', 'fips'])['residential'].shift(1)
ca_week['retail_and_recreation_lag'] = ca.groupby(['Year_week', 'fips'])['retail_and_recreation'].shift(1)
ca_week['workplaces_lag'] = ca.groupby(['Year_week', 'fips'])['workplaces'].shift(1)
ca_week.dropna(inplace=True)


In [40]:
X2 = ca_week[['ln_cases_lag']].dropna()
Y2 = ca_week.loc[X2.index, 'ln_cases']
X2 = sm.add_constant(X2)  # Add a constant term to the model
model2 = sm.OLS(Y2, X2).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               ln_cases   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.1280
Date:                Wed, 17 May 2023   Prob (F-statistic):              0.720
Time:                        22:41:04   Log-Likelihood:                -5202.5
No. Observations:                3306   AIC:                         1.041e+04
Df Residuals:                    3304   BIC:                         1.042e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            1.8627      0.026     72.519      0.000       1.812       1.913
ln_cases_lag    -0.0052      0.015     -0.358      0.720      -0.034       0.023
==============================================================================
Omnibus:                       34.917   Durbin-Watson:                   0.410
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               35.862
Skew:                           0.252   Prob(JB):                     1.63e-08
Kurtosis:                       2.923   Cond. No.                         2.56
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [41]:
X3 = ca_week[['transit_stations_lag', 'parks_lag', 'residential_lag', 'retail_and_recreation_lag', 'workplaces_lag',
              'unemployment_rate', 'some_college', 'bachelor_or_higher', 'high_school', 'less_than_high_school',
              'population_density',
              'poverty_rate', 'party_DEMOCRAT']].dropna()
X3 = sm.add_constant(X3)  # Add a constant term to the model
Y3 = ca_week.loc[X3.index, 'ln_cases']
model3 = sm.OLS(Y3, X3).fit()
model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               ln_cases   R-squared:                       0.237
Model:                            OLS   Adj. R-squared:                  0.235
Method:                 Least Squares   F-statistic:                     93.21
Date:                Wed, 17 May 2023   Prob (F-statistic):          1.30e-184
Time:                        22:41:04   Log-Likelihood:                -4754.6
No. Observations:                3306   AIC:                             9533.
Df Residuals:                    3294   BIC:                             9606.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
transit_stations_lag          0.0062      0.001      5.305      0.000       0.004       0.008
parks_lag                     0.0014      0.001      2.709      0.007       0.000       0.002
residential_lag              -0.0613      0.008     -7.415      0.000      -0.078      -0.045
retail_and_recreation_lag    -0.0411      0.002    -22.524      0.000      -0.045      -0.038
workplaces_lag               -0.0221      0.003     -6.609      0.000      -0.029      -0.016
unemployment_rate             0.0290      0.008      3.492      0.000       0.013       0.045
some_college                 -0.0097      0.005     -1.911      0.056      -0.020       0.000
bachelor_or_higher            0.0059      0.002      3.705      0.000       0.003       0.009
high_school                   0.0235      0.008      3.050      0.002       0.008       0.039
less_than_high_school         0.0193      0.004      4.372      0.000       0.011       0.028
population_density          -2.8e-06   7.61e-06     -0.368      0.713   -1.77e-05    1.21e-05
poverty_rate                 -0.0162      0.009     -1.835      0.067      -0.034       0.001
party_DEMOCRAT                0.0004   6.12e-05      6.361      0.000       0.000       0.001
==============================================================================
Omnibus:                       88.354   Durbin-Watson:                   0.635
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               94.869
Skew:                           0.410   Prob(JB):                     2.51e-21
Kurtosis:                       3.126   Cond. No.                     1.05e+19
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.88e-28. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [42]:
md_week = md.groupby(['Year_week', 'fips']).mean().reset_index()
md_week['ln_cases_lag'] = md.groupby(['Year_week', 'fips'])['ln_cases'].shift(1)
md_week['transit_stations_lag'] = md.groupby(['Year_week', 'fips'])['transit_stations'].shift(1)
md_week['parks_lag'] = md.groupby(['Year_week', 'fips'])['parks'].shift(1)
md_week['residential_lag'] = md.groupby(['Year_week', 'fips'])['residential'].shift(1)
md_week['retail_and_recreation_lag'] = md.groupby(['Year_week', 'fips'])['retail_and_recreation'].shift(1)
md_week['workplaces_lag'] = md.groupby(['Year_week', 'fips'])['workplaces'].shift(1)
md_week.dropna(inplace=True)

In [43]:
X2 = md_week[['ln_cases_lag']].dropna()
Y2 = md_week.loc[X2.index, 'ln_cases']
X2 = sm.add_constant(X2)  # Add a constant term to the model
model2 = sm.OLS(Y2, X2).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               ln_cases   R-squared:                       0.067
Model:                            OLS   Adj. R-squared:                  0.065
Method:                 Least Squares   F-statistic:                     34.92
Date:                Wed, 17 May 2023   Prob (F-statistic):           6.46e-09
Time:                        22:41:04   Log-Likelihood:                -709.28
No. Observations:                 488   AIC:                             1423.
Df Residuals:                     486   BIC:                             1431.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            1.7397      0.084     20.658      0.000       1.574       1.905
ln_cases_lag     0.2388      0.040      5.910      0.000       0.159       0.318
==============================================================================
Omnibus:                       11.862   Durbin-Watson:                   0.385
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               20.774
Skew:                           0.103   Prob(JB):                     3.08e-05
Kurtosis:                       3.990   Cond. No.                         4.37
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [44]:
X3 = md_week[['transit_stations_lag', 'parks_lag', 'residential_lag', 'retail_and_recreation_lag', 'workplaces_lag',
              'unemployment_rate', 'some_college', 'bachelor_or_higher', 'high_school', 'less_than_high_school',
              'population_density',
              'poverty_rate', 'party_DEMOCRAT']].dropna()
X3 = sm.add_constant(X3)  # Add a constant term to the model
Y3 = md_week.loc[X3.index, 'ln_cases']
model3 = sm.OLS(Y3, X3).fit()
model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               ln_cases   R-squared:                       0.219
Model:                            OLS   Adj. R-squared:                  0.201
Method:                 Least Squares   F-statistic:                     12.16
Date:                Wed, 17 May 2023   Prob (F-statistic):           2.90e-20
Time:                        22:41:04   Log-Likelihood:                -665.78
No. Observations:                 488   AIC:                             1356.
Df Residuals:                     476   BIC:                             1406.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
transit_stations_lag          0.0040      0.003      1.401      0.162      -0.002       0.010
parks_lag                    -0.0004      0.001     -0.459      0.647      -0.002       0.001
residential_lag               0.0448      0.022      2.062      0.040       0.002       0.088
retail_and_recreation_lag    -0.0331      0.005     -6.667      0.000      -0.043      -0.023
workplaces_lag                0.0094      0.009      1.033      0.302      -0.009       0.027
unemployment_rate            -0.3844      0.049     -7.864      0.000      -0.480      -0.288
some_college                  0.0873      0.029      3.041      0.002       0.031       0.144
bachelor_or_higher            0.0057      0.006      0.987      0.324      -0.006       0.017
high_school                  -0.0276      0.030     -0.909      0.364      -0.087       0.032
less_than_high_school         0.1157      0.032      3.645      0.000       0.053       0.178
population_density         3.505e-05   6.73e-05      0.520      0.603   -9.73e-05       0.000
poverty_rate                  0.0174      0.045      0.390      0.697      -0.070       0.105
party_DEMOCRAT                0.0018      0.000      5.655      0.000       0.001       0.002
==============================================================================
Omnibus:                        6.054   Durbin-Watson:                   0.604
Prob(Omnibus):                  0.048   Jarque-Bera (JB):                7.587
Skew:                           0.114   Prob(JB):                       0.0225
Kurtosis:                       3.567   Cond. No.                     2.08e+19
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 6.21e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [45]:
oh_week = oh.groupby(['Year_week', 'fips']).mean().reset_index()
oh_week['ln_cases_lag'] = oh.groupby(['Year_week', 'fips'])['ln_cases'].shift(1)
oh_week['transit_stations_lag'] = oh.groupby(['Year_week', 'fips'])['transit_stations'].shift(1)
oh_week['parks_lag'] = oh.groupby(['Year_week', 'fips'])['parks'].shift(1)
oh_week['residential_lag'] = oh.groupby(['Year_week', 'fips'])['residential'].shift(1)
oh_week['retail_and_recreation_lag'] = oh.groupby(['Year_week', 'fips'])['retail_and_recreation'].shift(1)
oh_week['workplaces_lag'] = oh.groupby(['Year_week', 'fips'])['workplaces'].shift(1)
oh_week.dropna(inplace=True)

In [46]:
X2 = oh_week[['ln_cases_lag']].dropna()
Y2 = oh_week.loc[X2.index, 'ln_cases']
X2 = sm.add_constant(X2)  # Add a constant term to the model
model2 = sm.OLS(Y2, X2).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               ln_cases   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                    0.1518
Date:                Wed, 17 May 2023   Prob (F-statistic):              0.697
Time:                        22:41:04   Log-Likelihood:                -467.34
No. Observations:                 260   AIC:                             938.7
Df Residuals:                     258   BIC:                             945.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            1.9436      0.133     14.603      0.000       1.681       2.206
ln_cases_lag     0.0309      0.079      0.390      0.697      -0.125       0.187
==============================================================================
Omnibus:                       52.005   Durbin-Watson:                   0.216
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               17.186
Skew:                           0.386   Prob(JB):                     0.000185
Kurtosis:                       2.006   Cond. No.                         2.99
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [47]:
X3 = oh_week[['transit_stations_lag', 'parks_lag', 'residential_lag', 'retail_and_recreation_lag', 'workplaces_lag',
              'unemployment_rate', 'some_college', 'bachelor_or_higher', 'high_school', 'less_than_high_school',
              'population_density',
              'poverty_rate', 'party_DEMOCRAT']].dropna()
X3 = sm.add_constant(X3)  # Add a constant term to the model
Y3 = oh_week.loc[X3.index, 'ln_cases']
model3 = sm.OLS(Y3, X3).fit()
model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               ln_cases   R-squared:                       0.123
Model:                            OLS   Adj. R-squared:                  0.085
Method:                 Least Squares   F-statistic:                     3.175
Date:                Wed, 17 May 2023   Prob (F-statistic):           0.000477
Time:                        22:41:04   Log-Likelihood:                -450.29
No. Observations:                 260   AIC:                             924.6
Df Residuals:                     248   BIC:                             967.3
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
transit_stations_lag         -0.0100      0.008     -1.249      0.213      -0.026       0.006
parks_lag                     0.0043      0.002      2.273      0.024       0.001       0.008
residential_lag               0.0356      0.040      0.898      0.370      -0.042       0.114
retail_and_recreation_lag    -0.0095      0.010     -0.954      0.341      -0.029       0.010
workplaces_lag               -0.0075      0.017     -0.450      0.653      -0.040       0.025
unemployment_rate            -0.1107      0.066     -1.689      0.092      -0.240       0.018
some_college                 -0.0265      0.025     -1.077      0.282      -0.075       0.022
bachelor_or_higher            0.0043      0.015      0.290      0.772      -0.025       0.033
high_school                   0.0206      0.025      0.824      0.411      -0.029       0.070
less_than_high_school         0.3125      0.106      2.952      0.003       0.104       0.521
population_density            0.0004      0.000      1.619      0.107    -7.9e-05       0.001
poverty_rate                 -0.1384      0.064     -2.161      0.032      -0.264      -0.012
party_DEMOCRAT                0.0031      0.001      3.227      0.001       0.001       0.005
==============================================================================
Omnibus:                       26.465   Durbin-Watson:                   0.391
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               19.457
Skew:                           0.561   Prob(JB):                     5.96e-05
Kurtosis:                       2.266   Cond. No.                     9.07e+18
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 5.16e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [48]:
pa_week = pa.groupby(['Year_week', 'fips']).mean().reset_index()
pa_week['ln_cases_lag'] = pa.groupby(['Year_week', 'fips'])['ln_cases'].shift(1)
pa_week['transit_stations_lag'] = pa.groupby(['Year_week', 'fips'])['transit_stations'].shift(1)
pa_week['parks_lag'] = pa.groupby(['Year_week', 'fips'])['parks'].shift(1)
pa_week['residential_lag'] = pa.groupby(['Year_week', 'fips'])['residential'].shift(1)
pa_week['retail_and_recreation_lag'] = pa.groupby(['Year_week', 'fips'])['retail_and_recreation'].shift(1)
pa_week['workplaces_lag'] = pa.groupby(['Year_week', 'fips'])['workplaces'].shift(1)
pa_week.dropna(inplace=True)

In [49]:
X2 = pa_week[['ln_cases_lag']].dropna()
Y2 = pa_week.loc[X2.index, 'ln_cases']
X2 = sm.add_constant(X2)  # Add a constant term to the model
model2 = sm.OLS(Y2, X2).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               ln_cases   R-squared:                       0.047
Model:                            OLS   Adj. R-squared:                  0.033
Method:                 Least Squares   F-statistic:                     3.367
Date:                Wed, 17 May 2023   Prob (F-statistic):             0.0709
Time:                        22:41:04   Log-Likelihood:                -100.63
No. Observations:                  70   AIC:                             205.3
Df Residuals:                      68   BIC:                             209.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            3.1549      0.158     19.987      0.000       2.840       3.470
ln_cases_lag     0.1640      0.089      1.835      0.071      -0.014       0.342
==============================================================================
Omnibus:                        7.434   Durbin-Watson:                   0.696
Prob(Omnibus):                  0.024   Jarque-Bera (JB):                6.823
Skew:                          -0.637   Prob(JB):                       0.0330
Kurtosis:                       3.848   Cond. No.                         2.59
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [50]:
X3 = pa_week[['transit_stations_lag', 'parks_lag', 'residential_lag', 'retail_and_recreation_lag', 'workplaces_lag',
              'unemployment_rate', 'some_college', 'bachelor_or_higher', 'high_school', 'less_than_high_school',
              'population_density',
              'poverty_rate', 'party_DEMOCRAT']].dropna()
X3 = sm.add_constant(X3)  # Add a constant term to the model
Y3 = pa_week.loc[X3.index, 'ln_cases']
model3 = sm.OLS(Y3, X3).fit()
model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               ln_cases   R-squared:                       0.573
Model:                            OLS   Adj. R-squared:                  0.493
Method:                 Least Squares   F-statistic:                     7.089
Date:                Wed, 17 May 2023   Prob (F-statistic):           1.86e-07
Time:                        22:41:04   Log-Likelihood:                -72.497
No. Observations:                  70   AIC:                             169.0
Df Residuals:                      58   BIC:                             196.0
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
transit_stations_lag          0.0170      0.008      2.253      0.028       0.002       0.032
parks_lag                     0.0058      0.002      3.130      0.003       0.002       0.009
residential_lag              -0.1114      0.044     -2.530      0.014      -0.200      -0.023
retail_and_recreation_lag    -0.0077      0.012     -0.619      0.539      -0.033       0.017
workplaces_lag               -0.0982      0.023     -4.323      0.000      -0.144      -0.053
unemployment_rate            -0.2293      0.073     -3.163      0.002      -0.374      -0.084
some_college                  0.0938      0.065      1.434      0.157      -0.037       0.225
bachelor_or_higher           -0.0122      0.016     -0.741      0.461      -0.045       0.021
high_school                   0.0162      0.035      0.457      0.649      -0.055       0.087
less_than_high_school        -0.0330      0.041     -0.797      0.429      -0.116       0.050
population_density        -6.412e-05   7.03e-05     -0.913      0.365      -0.000    7.65e-05
poverty_rate                  0.0731      0.062      1.181      0.243      -0.051       0.197
party_DEMOCRAT                0.0006      0.001      1.218      0.228      -0.000       0.002
==============================================================================
Omnibus:                        2.894   Durbin-Watson:                   1.157
Prob(Omnibus):                  0.235   Jarque-Bera (JB):                2.087
Skew:                          -0.362   Prob(JB):                        0.352
Kurtosis:                       3.437   Cond. No.                     1.91e+19
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.56e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

Comparing results from Dynamic Panel Data to PCA selected features in Part 1, the results here are a lot less desirable. This means the current procedure of performing Dynamic Panel Data Analysis is not appropriate for the objective.

In [52]:
ca_week['ln_cases'] = np.log(ca_week['daily_cases_per_100k'])
ca_week

KeyError: 'daily_cases_per_100k'